In [11]:
nodes= [:P, :W, :C, :R, :O, :F, :M]
supply = [:P, :W]
plant = [:C, :R]
city = [:O, :F, :M]
arcs=[(:P,:C), (:P,:R), (:W,:C), (:W,:R), 
      (:C,:O), (:C,:F), (:C,:M), 
      (:R,:O), (:R,:F), (:R,:M)]

R = Dict(zip(city, [3.5, 2.5, 2.85]))
B = Dict(zip(nodes, [400, 650, 0, 0, -550, -600, -700]))
C = Dict(zip(arcs, [0.35, 0.5, 0.3, 0.3, 0.6, 0.5, 0.25, 0.25, 0.25, 0.65]))
P = Dict(zip(plant, [0.25, 0.3]))
H = Dict(zip(plant, [900, 500]))
M = Dict(zip(city, [550, 600, 700]))

using JuMP, Cbc

m = Model()

@variable(m, x[arcs] >= 0)

@objective(m, Max, 3.5(x[(:C, :O)] + x[(:R, :O)]) + 2.5(x[(:C, :F)] + x[(:R, :F)]) + 2.85(x[(:C, :M)] + x[(:R, :M)]) - sum(C[i] * x[i] for i in arcs))

@constraint(m, limit_P, x[(:P,:C)] + x[(:P,:R)] <= 400)
@constraint(m, limit_W, x[(:W,:C)] + x[(:W,:R)] <= 650)
@constraint(m, balance_C, x[(:P,:C)] + x[(:W,:C)] - x[(:C,:O)] - x[(:C,:F)] - x[(:C,:M)] == 0)
@constraint(m, balance_R, x[(:P,:R)] + x[(:W,:R)] - x[(:R,:O)] - x[(:R,:F)] - x[(:R,:M)] == 0)
@constraint(m, limit_O, x[(:C,:O)] + x[(:R,:O)] <= 550)
@constraint(m, limit_F, x[(:C,:F)] + x[(:R,:F)] <= 600)
@constraint(m, limit_M, x[(:C,:M)] + x[(:R,:M)] <= 700)
@constraint(m, limit_C, x[(:P,:C)] + x[(:W,:C)] <= 900)
@constraint(m, limit_R, x[(:W,:R)] + x[(:P,:R)] <= 500)

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Presolve 9 (0) rows, 10 (0) columns and 24 (0) elements
0  Obj -0 Dual inf 15.199999 (6)
12  Obj 2735
Optimal - objective value 2735
Optimal objective 2735 - 12 iterations time 0.002


* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = unset (status_ will also be -1)
"

* Candidate solution
  Objective value      : 2735.0
  Objective bound      : 2735.0
  Primal solution :
    x[(:C, :F)] : 0.0
    x[(:C, :M)] : 500.0
    x[(:C, :O)] : 50.0
    x[(:P, :C)] : 400.0
    x[(:P, :R)] : 0.0
    x[(:R, :F)] : 0.0
    x[(:R, :M)] : 0.0
    x[(:R, :O)] : 500.0
    x[(:W, :C)] : 150.0
    x[(:W, :R)] : 500.0

* Work counters
  Solve time (sec)   : 0.00100
